### Imports

In [1]:
from definitions import *

### Make quiz prompt

In [2]:
with open("../../metrics/knowledge_tests/obrazovaka/oblomov.json", "r") as f:
    quiz = json.load(f)

In [3]:
part = quiz[0]
part

{'question': 'К какому литературному направлению относится роман Гончарова «Обломов»?',
 'answers': [{'answer': 'Романтизм;', 'is_correct': 0},
  {'answer': 'Реализм;', 'is_correct': 1},
  {'answer': 'Классицизм;', 'is_correct': 0},
  {'answer': 'Сентиментализм.', 'is_correct': 0}]}

In [4]:
def build_prompt(quiz_part):
    processed_answers = ""
    for i, answer in enumerate(quiz_part["answers"]):
        processed_answers += f"{i + 1}. {answer['answer']}\n"
    # return f"Выбери один верный вариант из предложенных.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"
    return f"Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"

In [5]:
print(build_prompt(part))

Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.
Вопрос: К какому литературному направлению относится роман Гончарова «Обломов»?
Варианты ответа:
1. Романтизм;
2. Реализм;
3. Классицизм;
4. Сентиментализм.



In [54]:
def build_prompt_with_summary(quiz_part, summary):
    processed_answers = ""
    for i, answer in enumerate(quiz_part["answers"]):
        processed_answers += f"{i + 1}. {answer['answer']}\n"
    # return f"Выбери один верный вариант из предложенных.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"
    return f"Внимательно прочитай краткое содержание книги, и потом ответь на вопрос.\nКраткое содержание:\n{summary}\nВыбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.\nВопрос: {quiz_part['question']}\nВарианты ответа:\n{processed_answers}"

In [55]:
print(build_prompt_with_summary(part, "kek"))

Внимательно прочитай краткое содержание книги, и потом ответь на вопрос.
Краткое содержание:
kek
Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.
Вопрос: В чем автор видит причины «обломовщины»?
Варианты ответа:
1. В деградации и застое чистой, нерасчетливой, нежной, но ленивой души героя;
2. В окружении героя, которое не давало ему развиваться;
3. В недостаточно хорошем образовании героя;
4. В отсутствии достойных стимулов к развитию.



### Learn to make requests to ollama

In [47]:
! ollama list

NAME                    	ID          	SIZE  	MODIFIED     
dolphin-phi:latest      	c5761fc77240	1.6 GB	14 hours ago	
gemma:2b                	b50d6c999e59	1.7 GB	7 hours ago 	
gemma:7b                	a72c7f4d0a15	5.0 GB	7 hours ago 	
llama2:13b              	d475bf4c50bc	7.4 GB	7 hours ago 	
llama2:70b              	e7f6c06ffef4	38 GB 	5 hours ago 	
llama2:latest           	78e26419b446	3.8 GB	14 hours ago	
llama2-uncensored:latest	44040b922233	3.8 GB	13 hours ago	
llama3:70b              	bcfb190ca3a7	39 GB 	6 hours ago 	
llama3:latest           	71a106a91016	4.7 GB	16 hours ago	
llava:latest            	8dd30f6b0cb1	4.7 GB	7 hours ago 	
mistral:latest          	61e88e884507	4.1 GB	14 hours ago	
neural-chat:latest      	89fa737d3b85	4.1 GB	14 hours ago	
orca-mini:latest        	2dbd9f439647	2.0 GB	7 hours ago 	
phi:latest              	e2fd6321a5fe	1.6 GB	14 hours ago	
solar:latest            	059fdabbe6e6	6.1 GB	7 hours ago 	
starling-lm:latest      	39153f619be6	4.1 GB	14 hours ag

In [13]:
def get_model_response(model, prompt):
    headers = {
        'Content-Type': 'application/json',
    }
    json_data = {
        'model': model,
        'prompt': prompt,
        'stream': False,
        "options": {
            "seed": 123,
            "temperature": 0
        }
    }

    response = requests.post('http://localhost:11434/api/generate', headers=headers, json=json_data)
    payload = json.loads(response.content)
    return payload["response"]

In [48]:
get_model_response("llama3", "say hi")

"Hi! How's it going?"

### Try to pass quiz with a model

In [16]:
prompt = build_prompt(part)
print(prompt)
get_model_response(model, prompt)

Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.
Вопрос: В чем автор видит причины «обломовщины»?
Варианты ответа:
1. В деградации и застое чистой, нерасчетливой, нежной, но ленивой души героя;
2. В окружении героя, которое не давало ему развиваться;
3. В недостаточно хорошем образовании героя;
4. В отсутствии достойных стимулов к развитию.



'1.'

In [17]:
len(quiz)

19

In [56]:
def eval_model_on_quiz(model, quiz, summary=None):
    num_ok = 0
    num_all = 0
    for part in tqdm(quiz):
        if summary:
            prompt = build_prompt_with_summary(part, summary)
        else:
            prompt = build_prompt(part)
        answer = get_model_response(model, prompt)
        first_part = answer.split()[0]
        if first_part.endswith("."):
            first_part = first_part[:-1]
        ok_answer = -1
        for i, x in enumerate(part["answers"]):
            if x["is_correct"] == 1:
                ok_answer = i + 1
        try:
            first_part = int(first_part)
        except Exception:
            print(f"Cannot convert model first part answer to int: '{answer}'")
        if ok_answer == first_part:
            num_ok += 1
        num_all += 1
    print(f"result is {num_ok} out of {num_all}. {num_ok / num_all * 100:.2f}%")
    return num_ok, num_all

In [59]:
models = ["llama2", "llama3"]
rows = []
for model in models:
    tests_dir = Path("../../metrics/knowledge_tests/obrazovaka/")
    for test_path in tests_dir.glob("*.json"):
        with open(test_path, "r") as f:
            quiz = json.load(f)
        with open(f"../../downloaded_books/briefly_summaries/very_short/{test_path.stem}.txt", "r") as f:
            summary = f.read()
        num_ok, num_all = eval_model_on_quiz(model, quiz, summary=summary)
        rows.append({
            "model": model,
            "test_path": test_path,
            "num_ok": num_ok,
            "num_all": num_all,
            "percent": float(f"{(num_ok / num_all * 100):.2f}"),
        })

100%|██████████| 16/16 [00:13<00:00,  1.18it/s]


result is 4 out of 16. 25.00%


 30%|███       | 3/10 [00:10<00:21,  3.12s/it]

Cannot convert model first part answer to int: 'Выбери один верный вариант из предложенных. В качестве ответа напиши только номер, без дополнительного текста. Например: '1', '2', '3', '4'.

Варианты ответа:

1. 1'


 50%|█████     | 5/10 [00:11<00:07,  1.59s/it]

...

result is 6 out of 12. 50.00%


100%|██████████| 16/16 [00:10<00:00,  1.47it/s]


result is 10 out of 16. 62.50%


100%|██████████| 10/10 [00:08<00:00,  1.14it/s]

result is 5 out of 10. 50.00%


In [43]:
# for row in rows:
    # row["percent"] = float(f'{(row["num_ok"] / row["num_all"] * 100):.2f}')

In [60]:
df = pd.DataFrame(rows)
df.to_csv("some_results_briefly.csv")
df

,model,test_path,num_ok,num_all,percent
0,llama2,../../metrics/knowledge_tests/obrazovaka/the_d...,4,16,25.00
1,llama2,../../metrics/knowledge_tests/obrazovaka/the_b...,1,10,10.00
2,llama2,../../metrics/knowledge_tests/obrazovaka/and_q...,3,18,16.67
3,llama2,../../metrics/knowledge_tests/obrazovaka/oblom...,2,19,10.53
4,llama2,../../metrics/knowledge_tests/obrazovaka/eveni...,2,10,20.00
5,llama2,../../metrics/knowledge_tests/obrazovaka/docto...,2,13,15.38
6,llama2,../../metrics/knowledge_tests/obrazovaka/war_a...,14,70,20.00
7,llama2,../../metrics/knowledge_tests/obrazovaka/the_m...,4,16,25.00
8,llama2,../../metrics/knowledge_tests/obrazovaka/dead_...,3,19,15.79
9,llama2,../../metrics/knowledge_tests/obrazovaka/the_g...,2,10,20.00


In [61]:
models = ["llama2", "llama3"]
rows_stupid = []
for model in models:
    tests_dir = Path("../../metrics/knowledge_tests/obrazovaka/")
    for test_path in tests_dir.glob("*.json"):
        with open(test_path, "r") as f:
            quiz = json.load(f)
        with open(f"../../downloaded_books/stupid_summaries/{test_path.stem}.txt", "r") as f:
            summary = f.read()
        num_ok, num_all = eval_model_on_quiz(model, quiz, summary=summary)
        rows_stupid.append({
            "model": model,
            "test_path": test_path,
            "num_ok": num_ok,
            "num_all": num_all,
            "percent": float(f"{(num_ok / num_all * 100):.2f}"),
        })
df_stupid = pd.DataFrame(rows_stupid)
df_stupid

100%|██████████| 16/16 [00:16<00:00,  1.04s/it]


result is 4 out of 16. 25.00%


 30%|███       | 3/10 [00:03<00:07,  1.04s/it]

Cannot convert model first part answer to int: 'Okay, I've read the summary, and my answer is:

1.'


100%|██████████| 10/10 [00:06<00:00,  1.58it/s]


...

 80%|████████  | 8/10 [00:06<00:01,  1.12it/s]

Cannot convert model first part answer to int: 'None of the above (книга не содержит информации о Левине и Ките) -> нет ответа из предложенных вариантов.'


100%|██████████| 10/10 [00:07<00:00,  1.35it/s]

result is 4 out of 10. 40.00%


,model,test_path,num_ok,num_all,percent
0,llama2,../../metrics/knowledge_tests/obrazovaka/the_d...,4,16,25.00
1,llama2,../../metrics/knowledge_tests/obrazovaka/the_b...,3,10,30.00
2,llama2,../../metrics/knowledge_tests/obrazovaka/and_q...,4,18,22.22
3,llama2,../../metrics/knowledge_tests/obrazovaka/oblom...,2,19,10.53
4,llama2,../../metrics/knowledge_tests/obrazovaka/eveni...,2,10,20.00
5,llama2,../../metrics/knowledge_tests/obrazovaka/docto...,3,13,23.08
6,llama2,../../metrics/knowledge_tests/obrazovaka/war_a...,16,70,22.86
7,llama2,../../metrics/knowledge_tests/obrazovaka/the_m...,5,16,31.25
8,llama2,../../metrics/knowledge_tests/obrazovaka/dead_...,7,19,36.84
9,llama2,../../metrics/knowledge_tests/obrazovaka/the_g...,1,10,10.00


In [62]:
df_stupid.to_csv("some_results_stupid.csv")

In [52]:
init_df = pd.read_csv("../../metrics/knowledge_tests/obrazovaka/full_results.csv")
full_df = pd.concat((init_df, df))
full_df

,model,test_path,num_ok,num_all,percent
0,llama2,../../metrics/knowledge_tests/obrazovaka/the_d...,3,16,18.75
1,llama2,../../metrics/knowledge_tests/obrazovaka/the_b...,1,10,10.00
2,llama2,../../metrics/knowledge_tests/obrazovaka/and_q...,6,18,33.33
3,llama2,../../metrics/knowledge_tests/obrazovaka/oblom...,7,19,36.84
4,llama2,../../metrics/knowledge_tests/obrazovaka/eveni...,1,10,10.00
5,llama2,../../metrics/knowledge_tests/obrazovaka/docto...,3,13,23.08
6,llama2,../../metrics/knowledge_tests/obrazovaka/war_a...,17,70,24.29
7,llama2,../../metrics/knowledge_tests/obrazovaka/the_m...,4,16,25.00
8,llama2,../../metrics/knowledge_tests/obrazovaka/dead_...,4,19,21.05
9,llama2,../../metrics/knowledge_tests/obrazovaka/the_g...,2,10,20.00


In [53]:
full_df.to_csv("../../metrics/knowledge_tests/obrazovaka/full_results.csv", index=False)